<a href="https://colab.research.google.com/github/Hpatterson01/best_places_to_eat/blob/main/pizzatime.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

## Introduction: Business Problem <a name="introduction"></a>

In ths project I will try to use location data to find the the state/city around the US that serve the most pizza for us pizza lovers. This report will be targeted for the travelers who love pizza but also want to know which state/city to visit where there are other things to do. 

They are many states around the world who provide plenty of pizza locations but I will focus on the states that have the most locations then dive down deeper into the cities that have other things to do. The final decision will be made by the pizza lovers options or what they like to do in the city that they are visiting. 

## Data <a name="data"></a>

Based on definition of our problem, factors that will influence our decission are:
* number of opitions the visitors have such as entertainment, bars, gyms or even other food spots. 
* Number of pizza locations in the state. 
* Most popular pizza names 

## Methodology <a name="methodology"></a>

In this project I will discuss my efforts on examining the states of the US that have the most pizza locations Becasue the pizza lovers will have the most options to choose from. 

The fist step was to gather the required data for the pizza locations in which I will clean up the data then determine which states have the most pizza locaions. 

Second step, I will futher examine the data with the states that are chosen to chose what cities are best within those states. 

third step, will be to use the foursquare data to determine the venues in the different cities within that state. 

finally step, will foucsed on those cities that have other options such as entertainment, bars, etc by clustering. 




# IMPORT LIBRARIES 

In [ ]:
 import pandas as pd
import numpy as np 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium 
from geopy.geocoders import Nominatim 
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# LOAD DATA

In [ ]:
from google.colab import files
data_to_load = files.upload()

Saving Pizza_Restaurants.csv to Pizza_Restaurants (2).csv


In [ ]:
import io
pizza = pd.read_csv(io.BytesIO(data_to_load['Pizza_Restaurants.csv']))

## Analysis <a name="analysis"></a>



In [ ]:
pizza.head()

,id,dateAdded,dateUpdated,address,categories,primaryCategories,city,country,keys,latitude,longitude,menuPageURL,menus.amountMax,menus.amountMin,menus.currency,menus.dateSeen,menus.description,menus.name,name,postalCode,priceRangeCurrency,priceRangeMin,priceRangeMax,province
0,AVz3Y-7h3D1zeR_xDAqm,2017-06-30T05:05:40Z,2019-05-01T15:43:09Z,4203 E Kiehl Ave,"Pizza,Restaurant,American restaurants,Pizza Pl...",Accommodation & Food Services,Sherwood,US,us/ar/sherwood/4203ekiehlave/-1051391616,34.832300,-92.183800,http://www.citysearch.com/profile/menu/1550074...,7.98,7.98,USD,"2018-05-01T04:25:37.197Z,2018-04-16T04:36:02.3...",NaN,Cheese Pizza,Shotgun Dans Pizza,72120,USD,0,25,AR
1,AVweGPFF_7pvs4fzAAzQ,2016-04-02T04:02:49Z,2019-05-01T15:27:50Z,25 E Camelback Rd,"Pizza,Pizza Place,Restaurants",Accommodation & Food Services,Phoenix,US,us/az/phoenix/25ecamelbackrd/-727422936,33.509266,-112.073044,NaN,6.00,6.00,USD,"2018-03-03T02:38:06.381Z,2018-01-18T20:18:10.0...",NaN,Pizza Cookie,Sauce Pizza Wine,85012,USD,0,25,AZ
2,AVwdRGa9_7pvs4fz4E3K,2016-03-03T18:39:49Z,2019-05-01T12:52:25Z,3703 Paxton Ave,"Restaurant,Pizza Place,Restaurants",Accommodation & Food Services,Cincinnati,US,us/oh/cincinnati/3703paxtonave/-619797122,39.144883,-84.432685,http://www.citysearch.com/profile/menu/8160503...,6.49,6.49,USD,"2018-04-10T07:58:34.585Z,2018-04-21T05:43:21.4...",a saucelessampcomma double cheese pizza with a...,Pizza Blanca,Mios Pizzeria,45209,USD,0,25,OH
3,AVwdX4psIN2L1WUfvJB1,2016-03-29T05:08:59Z,2019-05-01T12:52:20Z,30495 John R Rd,"Pizza,Carry-out food,Pizza Place,Restaurants",Accommodation & Food Services,Madison Heights,US,us/mi/madisonheights/30495johnrrd/-874863116,42.516669,-83.106630,NaN,5.99,5.99,USD,"2016-10-20T21:50:02Z,2016-03-29T05:08:59Z",NaN,Small Pizza,Hungry Howies Pizza,48071,USD,25,40,MI
4,AVwdaeTtkufWRAb55pSH,2016-03-31T02:34:04Z,2019-05-01T12:50:45Z,3600 Eastern Ave,"Pizza,American restaurants,Pizza Place,Pizza e...",Accommodation & Food Services,Baltimore,US,us/md/baltimore/3600easternave/-1270965359,39.286630,-76.566984,NaN,5.49,5.49,USD,2016-03-31T02:34:04Z,NaN,Pizza Sub,Spartan Pizzeria,21224,USD,0,25,MD


In [ ]:
pizza.shape

(10000, 24)

In [ ]:
pizza.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 24 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  10000 non-null  object 
 1   dateAdded           10000 non-null  object 
 2   dateUpdated         10000 non-null  object 
 3   address             10000 non-null  object 
 4   categories          10000 non-null  object 
 5   primaryCategories   10000 non-null  object 
 6   city                10000 non-null  object 
 7   country             10000 non-null  object 
 8   keys                10000 non-null  object 
 9   latitude            10000 non-null  float64
 10  longitude           10000 non-null  float64
 11  menuPageURL         1679 non-null   object 
 12  menus.amountMax     10000 non-null  float64
 13  menus.amountMin     10000 non-null  float64
 14  menus.currency      10000 non-null  object 
 15  menus.dateSeen      10000 non-null  object 
 16  menus

In [ ]:
pizza.describe()

,latitude,longitude,menus.amountMax,menus.amountMin,priceRangeMin,priceRangeMax
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,38.371895,-90.063800,14.032670,13.416902,4.655500,27.764200
std,4.573537,17.340379,17.607233,10.742678,9.828412,6.017526
min,21.421326,-157.802443,0.000000,0.000000,0.000000,7.000000
25%,34.422946,-104.801126,8.490000,8.000000,0.000000,25.000000
50%,40.118880,-82.909949,13.390000,12.990000,0.000000,25.000000
75%,40.905048,-75.194741,17.950000,16.990000,0.000000,25.000000
max,64.850500,-71.946170,1395.000000,243.000000,50.000000,55.000000


In [ ]:
# Summarize categorical features 
pizza.describe(include=['object'])

,id,dateAdded,dateUpdated,address,categories,primaryCategories,city,country,keys,menuPageURL,menus.currency,menus.dateSeen,menus.description,menus.name,name,postalCode,priceRangeCurrency,province
count,10000,10000,10000,10000,10000,10000,10000,10000,10000,1679,10000,10000,3718,10000,10000,9996,10000,10000
unique,2285,2214,2193,2278,456,8,1028,1,2285,336,1,6303,2963,4749,1827,1771,1,44
top,AVzA1VVjLD2H7whiWFMI,2017-06-19T14:50:25Z,2018-06-29T19:34:53Z,6501 Kissena Blvd,"Restaurant,Pizza Place",Accommodation & Food Services,New York,US,us/ny/flushing/6501kissenablvd/-1686688850,http://www.citysearch.com/profile/menu/1160924...,USD,2016-04-01T06:37:20Z,With cheese and tomato sauce.,Cheese Pizza,California Pizza Kitchen,10016,USD,NY
freq,68,72,68,68,2690,9909,655,10000,68,49,10000,21,48,299,261,102,10000,2640


# CLEAN DATA

FIND NULL VALUES

In [ ]:
pizza.isnull().sum()

id                       0
dateAdded                0
dateUpdated              0
address                  0
categories               0
primaryCategories        0
city                     0
country                  0
keys                     0
latitude                 0
longitude                0
menuPageURL           8321
menus.amountMax          0
menus.amountMin          0
menus.currency           0
menus.dateSeen           0
menus.description     6282
menus.name               0
name                     0
postalCode               4
priceRangeCurrency       0
priceRangeMin            0
priceRangeMax            0
province                 0
dtype: int64

In [ ]:
# look at all null rows to decide what I want to do with the data

pizza.loc[(pizza['menuPageURL'].isnull())]

,id,dateAdded,dateUpdated,address,categories,primaryCategories,city,country,keys,latitude,longitude,menuPageURL,menus.amountMax,menus.amountMin,menus.currency,menus.dateSeen,menus.description,menus.name,name,postalCode,priceRangeCurrency,priceRangeMin,priceRangeMax,province
1,AVweGPFF_7pvs4fzAAzQ,2016-04-02T04:02:49Z,2019-05-01T15:27:50Z,25 E Camelback Rd,"Pizza,Pizza Place,Restaurants",Accommodation & Food Services,Phoenix,US,us/az/phoenix/25ecamelbackrd/-727422936,33.509266,-112.073044,NaN,6.00,6.00,USD,"2018-03-03T02:38:06.381Z,2018-01-18T20:18:10.0...",NaN,Pizza Cookie,Sauce Pizza Wine,85012,USD,0,25,AZ
3,AVwdX4psIN2L1WUfvJB1,2016-03-29T05:08:59Z,2019-05-01T12:52:20Z,30495 John R Rd,"Pizza,Carry-out food,Pizza Place,Restaurants",Accommodation & Food Services,Madison Heights,US,us/mi/madisonheights/30495johnrrd/-874863116,42.516669,-83.106630,NaN,5.99,5.99,USD,"2016-10-20T21:50:02Z,2016-03-29T05:08:59Z",NaN,Small Pizza,Hungry Howies Pizza,48071,USD,25,40,MI
4,AVwdaeTtkufWRAb55pSH,2016-03-31T02:34:04Z,2019-05-01T12:50:45Z,3600 Eastern Ave,"Pizza,American restaurants,Pizza Place,Pizza e...",Accommodation & Food Services,Baltimore,US,us/md/baltimore/3600easternave/-1270965359,39.286630,-76.566984,NaN,5.49,5.49,USD,2016-03-31T02:34:04Z,NaN,Pizza Sub,Spartan Pizzeria,21224,USD,0,25,MD
5,AVwdaeTtkufWRAb55pSH,2016-03-31T02:34:04Z,2019-05-01T12:50:45Z,3600 Eastern Ave,"Pizza,American restaurants,Pizza Place,Pizza e...",Accommodation & Food Services,Baltimore,US,us/md/baltimore/3600easternave/-1270965359,39.286630,-76.566984,NaN,10.99,10.99,USD,2016-03-31T02:34:04Z,NaN,White Pizza,Spartan Pizzeria,21224,USD,0,25,MD
6,AVwckQhv_7pvs4fzwhpf,2016-04-22T02:28:48Z,2019-05-01T12:50:22Z,1834 Euclid Ave,Pizza Place,Accommodation & Food Services,Berkeley,US,us/ca/berkeley/1834euclidave/-1109825433,37.875496,-122.260345,NaN,4.99,4.99,USD,"2018-02-22T11:10:19.468Z,2018-01-22T03:13:03.3...",NaN,Two Topping Pizza,La Vals,94709,USD,0,25,CA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9992,AVwdKXFYByjofQCxmIyZ,2016-03-21T00:12:46Z,2018-01-08T10:25:27Z,1126 E Lovejoy St,Pizza Place,Accommodation & Food Services,Buffalo,US,us/ny/buffalo/1126elovejoyst/1921210585,42.889759,-78.806747,NaN,27.10,6.05,USD,"2017-12-11T12:44:00.000Z,2016-03-21T00:13:00.0...",NaN,White Pizza,Carbone's Pizzeria,14206,USD,0,25,NY
9993,AVwdKXFYByjofQCxmIyZ,2016-03-21T00:12:46Z,2018-01-08T10:25:27Z,1126 E Lovejoy St,Pizza Place,Accommodation & Food Services,Buffalo,US,us/ny/buffalo/1126elovejoyst/1921210585,42.889759,-78.806747,NaN,27.10,6.05,USD,"2017-12-11T12:44:00.000Z,2016-03-21T00:13:00.0...",NaN,Cheese Burger Pizza,Carbone's Pizzeria,14206,USD,0,25,NY
9994,AVwdKXFYByjofQCxmIyZ,2016-03-21T00:12:46Z,2018-01-08T10:25:27Z,1126 E Lovejoy St,Pizza Place,Accommodation & Food Services,Buffalo,US,us/ny/buffalo/1126elovejoyst/1921210585,42.889759,-78.806747,NaN,6.95,4.40,USD,2017-12-11T12:44:00.000Z,NaN,Pizza Sub,Carbone's Pizzeria,14206,USD,0,25,NY
9995,AVwdKXFYByjofQCxmIyZ,2016-03-21T00:12:46Z,2018-01-08T10:25:27Z,1126 E Lovejoy St,Pizza Place,Accommodation & Food Services,Buffalo,US,us/ny/buffalo/1126elovejoyst/1921210585,42.889759,-78.806747,NaN,37.10,7.10,USD,"2017-12-11T12:44:00.000Z,2016-03-21T00:13:00.0...",NaN,Super Steak Pizza,Carbone's Pizzeria,14206,USD,0,25,NY


TOO MANY NULL VALUES FOR MENUPAGESURL AND MENU DESCRIPTION WILL DELETE BOTH COLUMNS 

In [ ]:
pizza.drop(['menus.description','menuPageURL'], axis=1)

,id,dateAdded,dateUpdated,address,categories,primaryCategories,city,country,keys,latitude,longitude,menus.amountMax,menus.amountMin,menus.currency,menus.dateSeen,menus.name,name,postalCode,priceRangeCurrency,priceRangeMin,priceRangeMax,province
0,AVz3Y-7h3D1zeR_xDAqm,2017-06-30T05:05:40Z,2019-05-01T15:43:09Z,4203 E Kiehl Ave,"Pizza,Restaurant,American restaurants,Pizza Pl...",Accommodation & Food Services,Sherwood,US,us/ar/sherwood/4203ekiehlave/-1051391616,34.832300,-92.183800,7.98,7.98,USD,"2018-05-01T04:25:37.197Z,2018-04-16T04:36:02.3...",Cheese Pizza,Shotgun Dans Pizza,72120,USD,0,25,AR
1,AVweGPFF_7pvs4fzAAzQ,2016-04-02T04:02:49Z,2019-05-01T15:27:50Z,25 E Camelback Rd,"Pizza,Pizza Place,Restaurants",Accommodation & Food Services,Phoenix,US,us/az/phoenix/25ecamelbackrd/-727422936,33.509266,-112.073044,6.00,6.00,USD,"2018-03-03T02:38:06.381Z,2018-01-18T20:18:10.0...",Pizza Cookie,Sauce Pizza Wine,85012,USD,0,25,AZ
2,AVwdRGa9_7pvs4fz4E3K,2016-03-03T18:39:49Z,2019-05-01T12:52:25Z,3703 Paxton Ave,"Restaurant,Pizza Place,Restaurants",Accommodation & Food Services,Cincinnati,US,us/oh/cincinnati/3703paxtonave/-619797122,39.144883,-84.432685,6.49,6.49,USD,"2018-04-10T07:58:34.585Z,2018-04-21T05:43:21.4...",Pizza Blanca,Mios Pizzeria,45209,USD,0,25,OH
3,AVwdX4psIN2L1WUfvJB1,2016-03-29T05:08:59Z,2019-05-01T12:52:20Z,30495 John R Rd,"Pizza,Carry-out food,Pizza Place,Restaurants",Accommodation & Food Services,Madison Heights,US,us/mi/madisonheights/30495johnrrd/-874863116,42.516669,-83.106630,5.99,5.99,USD,"2016-10-20T21:50:02Z,2016-03-29T05:08:59Z",Small Pizza,Hungry Howies Pizza,48071,USD,25,40,MI
4,AVwdaeTtkufWRAb55pSH,2016-03-31T02:34:04Z,2019-05-01T12:50:45Z,3600 Eastern Ave,"Pizza,American restaurants,Pizza Place,Pizza e...",Accommodation & Food Services,Baltimore,US,us/md/baltimore/3600easternave/-1270965359,39.286630,-76.566984,5.49,5.49,USD,2016-03-31T02:34:04Z,Pizza Sub,Spartan Pizzeria,21224,USD,0,25,MD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,AVwdKXFYByjofQCxmIyZ,2016-03-21T00:12:46Z,2018-01-08T10:25:27Z,1126 E Lovejoy St,Pizza Place,Accommodation & Food Services,Buffalo,US,us/ny/buffalo/1126elovejoyst/1921210585,42.889759,-78.806747,37.10,7.10,USD,"2017-12-11T12:44:00.000Z,2016-03-21T00:13:00.0...",Super Steak Pizza,Carbone's Pizzeria,14206,USD,0,25,NY
9996,AVwdKXFYByjofQCxmIyZ,2016-03-21T00:12:46Z,2018-01-08T10:25:27Z,1126 E Lovejoy St,Pizza Place,Accommodation & Food Services,Buffalo,US,us/ny/buffalo/1126elovejoyst/1921210585,42.889759,-78.806747,10.10,10.10,USD,2017-12-11T12:44:00.000Z,Large Cheese & 1 Item Pizza,Carbone's Pizzeria,14206,USD,0,25,NY
9997,AVz6kzsQFcQ3k02bDhwQ,2017-06-30T19:56:11Z,2018-01-07T10:51:29Z,3641 E Main St,"Pizza Place,Restaurants",Accommodation & Food Services,Saint Charles,US,us/il/saintcharles/3641emainst/1336179832,41.921300,-88.275300,8.45,8.45,USD,"2017-12-10T18:55:00.000Z,2017-10-28T21:52:00.000Z","Individual 6"" Pepperoni Stuffed Deep Dish Pizza",Giordano's Pizza,60174,USD,0,25,IL
9998,AVz6kzsQFcQ3k02bDhwQ,2017-06-30T19:56:11Z,2018-01-07T10:51:29Z,3641 E Main St,"Pizza Place,Restaurants",Accommodation & Food Services,Saint Charles,US,us/il/saintcharles/3641emainst/1336179832,41.921300,-88.275300,8.45,8.45,USD,"2017-12-10T18:55:00.000Z,2017-10-28T21:52:00.000Z","Individual 6"" Cheese Stuffed Deep Dish Pizza",Giordano's Pizza,60174,USD,0,25,IL


CHECKING DROPPED COLUMNS

In [ ]:
pizza.columns

Index(['id', 'dateAdded', 'dateUpdated', 'address', 'categories',
       'primaryCategories', 'city', 'country', 'keys', 'latitude', 'longitude',
       'menuPageURL', 'menus.amountMax', 'menus.amountMin', 'menus.currency',
       'menus.dateSeen', 'menus.description', 'menus.name', 'name',
       'postalCode', 'priceRangeCurrency', 'priceRangeMin', 'priceRangeMax',
       'province'],
      dtype='object')

In [ ]:
# Decided to use the top 5 categories in dataset
pizza.categories.value_counts()

Restaurant,Pizza Place                                                                       2690
Pizza Place                                                                                  2209
Pizza,Restaurant,Pizza Place,Restaurants                                                      370
Italian Restaurant,Pizza Place                                                                312
Restaurant,Pizza Place,Restaurants                                                            293
                                                                                             ... 
Pizza Place,Pizza Place Bella Vista - Southwark                                                 1
Pizza Place,Caterers,Restaurants,Pizza Place Grand Crossing,Pizza Place Avalon Park,Pizza       1
Pizza Place,Restaurant,Restaurants,Pizza                                                        1
Pizza Place,Pizza Restaurant                                                                    1
Pizza Place North Ar

In [ ]:
pizza['categories'].value_counts().loc[lambda x : x > 292]

Restaurant,Pizza Place                      2690
Pizza Place                                 2209
Pizza,Restaurant,Pizza Place,Restaurants     370
Italian Restaurant,Pizza Place               312
Restaurant,Pizza Place,Restaurants           293
Name: categories, dtype: int64

In [ ]:
pizza_categorized = pizza[pizza['categories'].isin(['Restaurant,Pizza Place','Pizza Place','Pizza,Restaurant,Pizza Place,Restaurants','Italian Restaurant,Pizza Place','Restaurant,Pizza Place,Restaurants'])]
pizza_categorized.head()

,id,dateAdded,dateUpdated,address,categories,primaryCategories,city,country,keys,latitude,longitude,menuPageURL,menus.amountMax,menus.amountMin,menus.currency,menus.dateSeen,menus.description,menus.name,name,postalCode,priceRangeCurrency,priceRangeMin,priceRangeMax,province
2,AVwdRGa9_7pvs4fz4E3K,2016-03-03T18:39:49Z,2019-05-01T12:52:25Z,3703 Paxton Ave,"Restaurant,Pizza Place,Restaurants",Accommodation & Food Services,Cincinnati,US,us/oh/cincinnati/3703paxtonave/-619797122,39.144883,-84.432685,http://www.citysearch.com/profile/menu/8160503...,6.49,6.49,USD,"2018-04-10T07:58:34.585Z,2018-04-21T05:43:21.4...",a saucelessampcomma double cheese pizza with a...,Pizza Blanca,Mios Pizzeria,45209,USD,0,25,OH
6,AVwckQhv_7pvs4fzwhpf,2016-04-22T02:28:48Z,2019-05-01T12:50:22Z,1834 Euclid Ave,Pizza Place,Accommodation & Food Services,Berkeley,US,us/ca/berkeley/1834euclidave/-1109825433,37.875496,-122.260345,NaN,4.99,4.99,USD,"2018-02-22T11:10:19.468Z,2018-01-22T03:13:03.3...",NaN,Two Topping Pizza,La Vals,94709,USD,0,25,CA
7,AVwckQhv_7pvs4fzwhpf,2016-04-22T02:28:48Z,2019-05-01T12:50:22Z,1834 Euclid Ave,Pizza Place,Accommodation & Food Services,Berkeley,US,us/ca/berkeley/1834euclidave/-1109825433,37.875496,-122.260345,NaN,5.39,5.39,USD,"2018-02-22T11:10:19.468Z,2018-01-22T03:13:03.3...",NaN,Three Or More Topping Pizza,La Vals,94709,USD,0,25,CA
8,AVwcvnln_7pvs4fzyfqZ,2016-04-01T06:35:24Z,2019-05-01T12:49:06Z,4819 Kerry Forest Pkwy,Pizza Place,Accommodation & Food Services,Tallahassee,US,us/fl/tallahassee/4819kerryforestpkwy/879863405,30.537097,-84.215156,NaN,2.50,2.50,USD,"2017-10-18T11:36:00.000Z,2018-01-22T20:32:19.1...",NaN,Pizza,Brickyard Pizza,32309,USD,0,25,FL
9,AVwcvnln_7pvs4fzyfqZ,2016-04-01T06:35:24Z,2019-05-01T12:49:06Z,4819 Kerry Forest Pkwy,Pizza Place,Accommodation & Food Services,Tallahassee,US,us/fl/tallahassee/4819kerryforestpkwy/879863405,30.537097,-84.215156,NaN,11.99,11.99,USD,"2017-10-18T11:36:00.000Z,2018-01-22T20:32:19.1...",NaN,Pizza,Brickyard Pizza,32309,USD,0,25,FL


In [ ]:
# filtering top 5 categories 
#pizza_categorized = pizza[pizza['categories'].isin([['Restaurant,Pizza Place'],'Pizza Place',['Pizza,Restaurant,Pizza Place,Restaurants'],['Italian Restaurant,Pizza Place'],['Restaurant,Pizza Place,Restaurants']])]
#pizza_categorized.head()

In [ ]:
# Checking dataframe to see if results are correct 
pizza_categorized['categories'].unique()

array(['Restaurant,Pizza Place,Restaurants', 'Pizza Place',
       'Pizza,Restaurant,Pizza Place,Restaurants',
       'Restaurant,Pizza Place', 'Italian Restaurant,Pizza Place'],
      dtype=object)

In [ ]:
pizza_categorized.categories.value_counts()

Restaurant,Pizza Place                      2690
Pizza Place                                 2209
Pizza,Restaurant,Pizza Place,Restaurants     370
Italian Restaurant,Pizza Place               312
Restaurant,Pizza Place,Restaurants           293
Name: categories, dtype: int64

### DROPPING MORE COLUMNS THAT i WILL NOT NEED

In [ ]:
pizza_categorized = pizza_categorized.drop(['dateAdded','dateUpdated','keys','menuPageURL','menus.dateSeen','menus.description'], axis=1)
pizza_categorized.head(10)

,id,address,categories,primaryCategories,city,country,latitude,longitude,menus.amountMax,menus.amountMin,menus.currency,menus.name,name,postalCode,priceRangeCurrency,priceRangeMin,priceRangeMax,province
2,AVwdRGa9_7pvs4fz4E3K,3703 Paxton Ave,"Restaurant,Pizza Place,Restaurants",Accommodation & Food Services,Cincinnati,US,39.144883,-84.432685,6.49,6.49,USD,Pizza Blanca,Mios Pizzeria,45209,USD,0,25,OH
6,AVwckQhv_7pvs4fzwhpf,1834 Euclid Ave,Pizza Place,Accommodation & Food Services,Berkeley,US,37.875496,-122.260345,4.99,4.99,USD,Two Topping Pizza,La Vals,94709,USD,0,25,CA
7,AVwckQhv_7pvs4fzwhpf,1834 Euclid Ave,Pizza Place,Accommodation & Food Services,Berkeley,US,37.875496,-122.260345,5.39,5.39,USD,Three Or More Topping Pizza,La Vals,94709,USD,0,25,CA
8,AVwcvnln_7pvs4fzyfqZ,4819 Kerry Forest Pkwy,Pizza Place,Accommodation & Food Services,Tallahassee,US,30.537097,-84.215156,2.50,2.50,USD,Pizza,Brickyard Pizza,32309,USD,0,25,FL
9,AVwcvnln_7pvs4fzyfqZ,4819 Kerry Forest Pkwy,Pizza Place,Accommodation & Food Services,Tallahassee,US,30.537097,-84.215156,11.99,11.99,USD,Pizza,Brickyard Pizza,32309,USD,0,25,FL
17,AVwc2xOokufWRAb5zu4_,965 N Resler Dr,"Restaurant,Pizza Place,Restaurants",Accommodation & Food Services,El Paso,US,31.862409,-106.558431,5.00,5.00,USD,HotNReady Large Pizza,Little Caesars Pizza,79912,USD,0,25,TX
18,AVwcr0iskufWRAb5x6lr,8111 Rowlock Way,"Pizza,Restaurant,Pizza Place,Restaurants",Accommodation & Food Services,Raleigh,US,35.884857,-78.737707,11.99,11.99,USD,"15"" Plain Cheese Pizza",Tonys Pizza,27613,USD,0,25,NC
20,AVwedtwTIN2L1WUf5Kjd,4024 Canal St,Pizza Place,Accommodation & Food Services,New Orleans,US,29.974032,-90.100317,12.25,12.25,USD,Bacon & Blue Cheese Pizza,Theos Neighborhood Pizza,70119,USD,25,40,LA
21,AVwedtwTIN2L1WUf5Kjd,4024 Canal St,Pizza Place,Accommodation & Food Services,New Orleans,US,29.974032,-90.100317,12.25,12.25,USD,Chicken Pesto Pizza,Theos Neighborhood Pizza,70119,USD,25,40,LA
22,AVwedtwTIN2L1WUf5Kjd,4024 Canal St,Pizza Place,Accommodation & Food Services,New Orleans,US,29.974032,-90.100317,13.50,13.50,USD,Big Dave's Buffalo Chicken Pizza,Theos Neighborhood Pizza,70119,USD,25,40,LA


# CHECKING OUT PROVINCE, MENUS.NAME, NAME

### EXAMINE PROVINCE

In [ ]:
pizza_categorized.province.value_counts()

NY    1378
CA     710
PA     542
FL     386
TX     325
IL     238
MI     235
OH     192
IN     165
NC     162
AZ     143
NV     128
VA     127
GA     116
WA     110
WI     101
CO      86
UT      84
MN      80
OR      80
MD      65
MO      62
KY      51
SC      50
NM      37
TN      36
KS      30
AR      23
IA      23
OK      22
NE      17
WV      17
LA      16
DE      16
AL       5
MS       4
ID       3
ND       3
SD       3
HI       2
AK       1
Name: province, dtype: int64

DECIDED TO COMPARE JUST NY AND CA. EAST VS WEST

In [ ]:
pizza_categorized = pizza_categorized[pizza_categorized['province'].isin(['NY','CA'])]
pizza_categorized.head()

,id,address,categories,primaryCategories,city,country,latitude,longitude,menus.amountMax,menus.amountMin,menus.currency,menus.name,name,postalCode,priceRangeCurrency,priceRangeMin,priceRangeMax,province
6,AVwckQhv_7pvs4fzwhpf,1834 Euclid Ave,Pizza Place,Accommodation & Food Services,Berkeley,US,37.875496,-122.260345,4.99,4.99,USD,Two Topping Pizza,La Vals,94709,USD,0,25,CA
7,AVwckQhv_7pvs4fzwhpf,1834 Euclid Ave,Pizza Place,Accommodation & Food Services,Berkeley,US,37.875496,-122.260345,5.39,5.39,USD,Three Or More Topping Pizza,La Vals,94709,USD,0,25,CA
51,AVwd3vBcIN2L1WUfz8Kd,786 Main St,"Restaurant,Pizza Place,Restaurants",Accommodation & Food Services,Poughkeepsie,US,41.696049,-73.901679,4.99,4.99,USD,All Meat Stuffed Pizza Slice,Tonys Pizza Pit,12603,USD,0,25,NY
52,AVwd3vBcIN2L1WUfz8Kd,786 Main St,"Restaurant,Pizza Place,Restaurants",Accommodation & Food Services,Poughkeepsie,US,41.696049,-73.901679,18.99,18.99,USD,Hawaiian Pizza,Tonys Pizza Pit,12603,USD,0,25,NY
53,AVwd3vBcIN2L1WUfz8Kd,786 Main St,"Restaurant,Pizza Place,Restaurants",Accommodation & Food Services,Poughkeepsie,US,41.696049,-73.901679,18.99,18.99,USD,4Cheese Pizza,Tonys Pizza Pit,12603,USD,0,25,NY


CHECKING RESULTS 

In [ ]:
pizza_categorized['province'].value_counts()

NY    1378
CA     710
Name: province, dtype: int64

### EXAMINE NAME

In [ ]:
# popular pizza spots 
pizza_categorized.name.value_counts()

California Pizza Kitchen           102
Gino's Pizzeria                     68
Mario's Pizzeria                    42
Butera's                            37
La Nova Pizzeria                    35
                                  ... 
Love At First Slice                  1
Sammy's Woodfired Pizza & Grill      1
Verrazano Pizza                      1
Tonino Pizzeria Contorni             1
Fat Tony's Pizza                     1
Name: name, Length: 338, dtype: int64

DECIDED TO KEEP TOP 5 PIZZA NAME

In [ ]:
pizza_categorized = pizza_categorized[pizza_categorized['name'].isin(['California Pizza Kitchen',"Gino's Pizzeria","Mario's Pizzeria","Butera's",'La Nova Pizzeria'])]
pizza_categorized.name.value_counts()

California Pizza Kitchen    102
Gino's Pizzeria              68
Mario's Pizzeria             42
Butera's                     37
La Nova Pizzeria             35
Name: name, dtype: int64

### EXAMINE MENUS.NAME

KEEPING ALL MENUS.NAME

In [ ]:
pizza_categorized['menus.name'].value_counts()

Hawaiian Pizza                                   11
Cheese Pizza                                      8
Taco Pizza                                        7
White Pizza                                       6
Veggie Pizza                                      4
                                                 ..
California Veggie Pizza ampamp Soup Lunch Duo     1
Artichoke, Sun Dried Tomato & Spinach Pizza       1
Spinach, Mushroom & Cheese Pizza                  1
Gluten Free 1 Topping Pizza                       1
Spinach Pizza Slice                               1
Name: menus.name, Length: 192, dtype: int64

#### LETS LOOK AT PRIMARY CATEGORIES 

In [ ]:
pizza_categorized['primaryCategories'].value_counts()

Accommodation & Food Services    284
Name: primaryCategories, dtype: int64

### LET'S LOOK AT THE CITIES

In [ ]:
# looks like flushing NY and Los Angles have the most pizza spots 
city_data = pizza_categorized['city'].value_counts().reset_index()
city_data.columns = ['city','values']
city_data

,city,values
0,Flushing,68
1,Los Angeles,47
2,Thousand Oaks,39
3,Hamburg,37
4,Buffalo,35
5,Seaford,24
6,Chappaqua,18
7,Newport Beach,14
8,Palo Alto,2


# EXAMINE NY

In [ ]:
pizza_categorized.head(1)

,id,address,categories,primaryCategories,city,country,latitude,longitude,menus.amountMax,menus.amountMin,menus.currency,menus.name,name,postalCode,priceRangeCurrency,priceRangeMin,priceRangeMax,province
4660,AVwc9VhBIN2L1WUfqvgD,2200 E Thousand Oaks Blvd,"Restaurant,Pizza Place",Accommodation & Food Services,Thousand Oaks,US,34.174461,-118.844574,14.39,14.39,USD,Catering Shrimp Scampi Pizza,California Pizza Kitchen,91362,USD,25,40,CA


In [ ]:
# Gather data for New York
ny_data = pizza_categorized[pizza_categorized['province'] == 'NY'].reset_index(drop=True)
ny_data.head(5)

,id,address,categories,primaryCategories,city,country,latitude,longitude,menus.amountMax,menus.amountMin,menus.currency,menus.name,name,postalCode,priceRangeCurrency,priceRangeMin,priceRangeMax,province
0,AVwcxeT5ByjofQCxiCJs,371 W Ferry St,"Restaurant,Pizza Place",Accommodation & Food Services,Buffalo,US,42.915339,-78.887454,4.99,4.99,USD,2 Slices of Cheese & Pepperoni Pizza,La Nova Pizzeria,14213,USD,0,25,NY
1,AVwcxeT5ByjofQCxiCJs,371 W Ferry St,"Restaurant,Pizza Place",Accommodation & Food Services,Buffalo,US,42.915339,-78.887454,15.49,15.49,USD,Garlic Lover's Pizza,La Nova Pizzeria,14213,USD,0,25,NY
2,AVwcxeT5ByjofQCxiCJs,371 W Ferry St,"Restaurant,Pizza Place",Accommodation & Food Services,Buffalo,US,42.915339,-78.887454,7.99,7.99,USD,"Pizza Sub, Cheese & Pepperoni",La Nova Pizzeria,14213,USD,0,25,NY
3,AVwcxeT5ByjofQCxiCJs,371 W Ferry St,"Restaurant,Pizza Place",Accommodation & Food Services,Buffalo,US,42.915339,-78.887454,16.99,16.99,USD,Chicken Bacon Ranch Pizza,La Nova Pizzeria,14213,USD,0,25,NY
4,AVwcxeT5ByjofQCxiCJs,371 W Ferry St,"Restaurant,Pizza Place",Accommodation & Food Services,Buffalo,US,42.915339,-78.887454,2.70,2.70,USD,La Nova Original White Pizza Nova,La Nova Pizzeria,14213,USD,0,25,NY


In [ ]:
ny_data.shape

(182, 18)

In [ ]:
address = 'New York'

geolocator = Nominatim(user_agent="New_york_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York are 40.7127281, -74.0060152.


In [ ]:
# create map of New York using latitude and longitude values

newyork = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(ny_data['latitude'],ny_data['longitude'],ny_data['city']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(cali)  
    
newyork

#EXAMINE CALIFORNIA

In [ ]:
# Gather data for California
ca_data = pizza_categorized[pizza_categorized['province'] == 'CA'].reset_index(drop=True)
ca_data.head(5)

,id,address,categories,primaryCategories,city,country,latitude,longitude,menus.amountMax,menus.amountMin,menus.currency,menus.name,name,postalCode,priceRangeCurrency,priceRangeMin,priceRangeMax,province
0,AVwc9VhBIN2L1WUfqvgD,2200 E Thousand Oaks Blvd,"Restaurant,Pizza Place",Accommodation & Food Services,Thousand Oaks,US,34.174461,-118.844574,14.39,14.39,USD,Catering Shrimp Scampi Pizza,California Pizza Kitchen,91362,USD,25,40,CA
1,AVwc9VhBIN2L1WUfqvgD,2200 E Thousand Oaks Blvd,"Restaurant,Pizza Place",Accommodation & Food Services,Thousand Oaks,US,34.174461,-118.844574,11.95,11.95,USD,Mushroom Pepperoni Sausage Pizza ampamp Soup L...,California Pizza Kitchen,91362,USD,25,40,CA
2,AVwc9VhBIN2L1WUfqvgD,2200 E Thousand Oaks Blvd,"Restaurant,Pizza Place",Accommodation & Food Services,Thousand Oaks,US,34.174461,-118.844574,14.49,14.49,USD,Tostada Pizza,California Pizza Kitchen,91362,USD,25,40,CA
3,AVwc9VhBIN2L1WUfqvgD,2200 E Thousand Oaks Blvd,"Restaurant,Pizza Place",Accommodation & Food Services,Thousand Oaks,US,34.174461,-118.844574,17.99,17.99,USD,Mushroom Pepperoni Sausage GlutenFree Pizza,California Pizza Kitchen,91362,USD,25,40,CA
4,AVwc9VhBIN2L1WUfqvgD,2200 E Thousand Oaks Blvd,"Restaurant,Pizza Place",Accommodation & Food Services,Thousand Oaks,US,34.174461,-118.844574,16.79,16.79,USD,Pepperoni GlutenFree Pizza,California Pizza Kitchen,91362,USD,25,40,CA


In [ ]:
ca_data.shape

(102, 18)

In [ ]:
address = 'California'

geolocator = Nominatim(user_agent="California_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of California are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of California are 36.7014631, -118.755997.


In [ ]:
# create map of California using latitude and longitude values

cali = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(ca_data['latitude'],ca_data['longitude'],ca_data['city']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(cali)  
    
cali

# DEFINE FOURSQUARE DATA

In [ ]:
CLIENT_ID = 'QLAGPXRLFTYIYN3DS1NQDQEMN2ZF5V1X1YWWH2U1BMZUMOH5' # your Foursquare ID
CLIENT_SECRET = '4KONB0F4TGNWFMRNSTNYYVHNOTJCLTDANXONK01IV0UBGA2Y' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: QLAGPXRLFTYIYN3DS1NQDQEMN2ZF5V1X1YWWH2U1BMZUMOH5
CLIENT_SECRET:4KONB0F4TGNWFMRNSTNYYVHNOTJCLTDANXONK01IV0UBGA2Y


### GET CITY NAMES

In [ ]:
ny_data.loc[0, 'city']

'Buffalo'

### GET CITY LATITUDE AND LONGITUDE

In [ ]:
city_latitude = ny_data.loc[0, 'latitude'] # city latitude value
city_longitude = ny_data.loc[0, 'longitude'] # city longitude value

city_name = ny_data.loc[0, 'city'] # city name

print('latitude and longitude values of {} are {}, {}.'.format(city_name, 
                                                               city_latitude, 
                                                               city_longitude))

latitude and longitude values of Buffalo are 42.915339, -78.887454.


#### Now, let's get the top 100 venues that are in Buffalo within a radius of 500 meters.

In [ ]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    city_latitude, 
    city_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=QLAGPXRLFTYIYN3DS1NQDQEMN2ZF5V1X1YWWH2U1BMZUMOH5&client_secret=4KONB0F4TGNWFMRNSTNYYVHNOTJCLTDANXONK01IV0UBGA2Y&v=20180605&ll=42.915339,-78.887454&radius=500&limit=100'

#### SEND GET REQUESTS 

In [ ]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '606baa4804959b5a1f06370d'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4b33ef50f964a520492225e3-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/pizza_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d1ca941735',
         'name': 'Pizza Place',
         'pluralName': 'Pizza Places',
         'primary': True,
         'shortName': 'Pizza'}],
       'delivery': {'id': '2226397',
        'provider': {'icon': {'name': '/delivery_provider_grubhub_20180129.png',
          'prefix': 'https://fastly.4sqi.net/img/general/cap/',
          'sizes': [40, 50]},
         'name': 'grubhub'},
        'url': 'https://www.grubhub.com/restaurant/la-nova-pizzeria-371-w-ferry-st-buffalo/2226397?affiliate=1131&utm_source=foursquare-af

In [ ]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [ ]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,La Nova Pizzeria,Pizza Place,42.915256,-78.888017
1,West Side Bazaar,Market,42.914885,-78.890332
2,The Left Bank,New American Restaurant,42.912842,-78.883014
3,Pho Dollar,Vietnamese Restaurant,42.915462,-78.889808
4,Remedy House,Coffee Shop,42.911403,-78.885872


In [ ]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

17 venues were returned by Foursquare.


### FUNCTION FOR ALL DATA

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['city', 
                  'city Latitude', 
                  'city Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
new_york_venues = getNearbyVenues(names=ny_data['city'],
                                   latitudes=ny_data['latitude'],
                                   longitudes=ny_data['longitude']
                                  )

Buffalo
Buffalo
Buffalo
Buffalo
Buffalo
Buffalo
Buffalo
Buffalo
Buffalo
Buffalo
Buffalo
Buffalo
Buffalo
Buffalo
Buffalo
Buffalo
Buffalo
Buffalo
Buffalo
Buffalo
Buffalo
Buffalo
Buffalo
Buffalo
Buffalo
Buffalo
Buffalo
Buffalo
Buffalo
Buffalo
Buffalo
Buffalo
Buffalo
Buffalo
Buffalo
Flushing
Flushing
Flushing
Flushing
Flushing
Flushing
Flushing
Flushing
Flushing
Flushing
Flushing
Flushing
Flushing
Flushing
Flushing
Flushing
Flushing
Flushing
Flushing
Flushing
Flushing
Flushing
Flushing
Flushing
Flushing
Flushing
Flushing
Flushing
Flushing
Flushing
Flushing
Flushing
Flushing
Flushing
Flushing
Flushing
Flushing
Flushing
Flushing
Flushing
Flushing
Flushing
Flushing
Flushing
Flushing
Flushing
Flushing
Flushing
Flushing
Flushing
Flushing
Flushing
Flushing
Flushing
Flushing
Flushing
Flushing
Flushing
Flushing
Flushing
Flushing
Flushing
Flushing
Flushing
Flushing
Flushing
Flushing
Flushing
Seaford
Seaford
Seaford
Seaford
Seaford
Seaford
Seaford
Seaford
Seaford
Seaford
Seaford
Seaford
Seaford
Seaf

#### SIZE OF DATAFRAME

In [ ]:
print(new_york_venues.shape)
new_york_venues.head(10)

(3552, 7)


,city,city Latitude,city Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Buffalo,42.915339,-78.887454,La Nova Pizzeria,42.915256,-78.888017,Pizza Place
1,Buffalo,42.915339,-78.887454,West Side Bazaar,42.914885,-78.890332,Market
2,Buffalo,42.915339,-78.887454,The Left Bank,42.912842,-78.883014,New American Restaurant
3,Buffalo,42.915339,-78.887454,Pho Dollar,42.915462,-78.889808,Vietnamese Restaurant
4,Buffalo,42.915339,-78.887454,Remedy House,42.911403,-78.885872,Coffee Shop
5,Buffalo,42.915339,-78.887454,Evolation Yoga,42.912403,-78.884132,Yoga Studio
6,Buffalo,42.915339,-78.887454,Essex St Pub,42.913226,-78.882527,Dive Bar
7,Buffalo,42.915339,-78.887454,Dollar General,42.915145,-78.889693,Discount Store
8,Buffalo,42.915339,-78.887454,Rite Aid,42.914674,-78.890765,Pharmacy
9,Buffalo,42.915339,-78.887454,7-Eleven,42.918804,-78.890230,Convenience Store


How many venus returned for each city

In [ ]:
new_york_venues.groupby('city').count()

,city Latitude,city Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
city,,,,,,
Buffalo,675,675,675,675,675,675
Chappaqua,540,540,540,540,540,540
Flushing,1224,1224,1224,1224,1224,1224
Hamburg,777,777,777,777,777,777
Seaford,336,336,336,336,336,336


how many unique categories can be curated from all the returned venues


In [ ]:
print('There are {} uniques categories.'.format(len(new_york_venues['Venue Category'].unique())))

There are 76 uniques categories.


# ANALYZE EACH CITY

In [ ]:
# one hot encoding
ny_onehot = pd.get_dummies(new_york_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
ny_onehot['city'] = new_york_venues['city'] 

# move neighborhood column to the first column
fixed_columns = [ny_onehot.columns[-1]] + list(ny_onehot.columns[:-1])
ny_onehot = ny_onehot[fixed_columns]

ny_onehot.head()

,city,American Restaurant,Art Gallery,Asian Restaurant,Bagel Shop,Bakery,Bar,Bookstore,Boutique,Bowling Alley,Burger Joint,Café,Cantonese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,College Gym,Concert Hall,Convenience Store,Creperie,Dance Studio,Deli / Bodega,Discount Store,Dive Bar,Donut Shop,Farmers Market,Fast Food Restaurant,Flower Shop,Food,Food Truck,Fountain,French Restaurant,Garden Center,Gas Station,Gastropub,Gym,Gymnastics Gym,Hardware Store,Hobby Shop,Hotel,Ice Cream Shop,Intersection,Italian Restaurant,Jewelry Store,Juice Bar,Market,Mexican Restaurant,Movie Theater,Museum,New American Restaurant,Noodle House,Other Repair Shop,Paper / Office Supplies Store,Performing Arts Venue,Pharmacy,Pizza Place,Pub,Restaurant,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Smoke Shop,Soccer Field,Spa,Spanish Restaurant,Sporting Goods Shop,Supermarket,Sushi Restaurant,Taco Place,Tennis Court,Thai Restaurant,Toy / Game Store,Train Station,Trattoria/Osteria,Vietnamese Restaurant,Wine Shop,Yoga Studio
0,Buffalo,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Buffalo,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Buffalo,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Buffalo,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,Buffalo,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
ny_onehot.shape

(3552, 77)

#### Grouping rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [ ]:
ny_grouped = ny_onehot.groupby('city').mean().reset_index()
ny_grouped

,city,American Restaurant,Art Gallery,Asian Restaurant,Bagel Shop,Bakery,Bar,Bookstore,Boutique,Bowling Alley,Burger Joint,Café,Cantonese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,College Gym,Concert Hall,Convenience Store,Creperie,Dance Studio,Deli / Bodega,Discount Store,Dive Bar,Donut Shop,Farmers Market,Fast Food Restaurant,Flower Shop,Food,Food Truck,Fountain,French Restaurant,Garden Center,Gas Station,Gastropub,Gym,Gymnastics Gym,Hardware Store,Hobby Shop,Hotel,Ice Cream Shop,Intersection,Italian Restaurant,Jewelry Store,Juice Bar,Market,Mexican Restaurant,Movie Theater,Museum,New American Restaurant,Noodle House,Other Repair Shop,Paper / Office Supplies Store,Performing Arts Venue,Pharmacy,Pizza Place,Pub,Restaurant,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Smoke Shop,Soccer Field,Spa,Spanish Restaurant,Sporting Goods Shop,Supermarket,Sushi Restaurant,Taco Place,Tennis Court,Thai Restaurant,Toy / Game Store,Train Station,Trattoria/Osteria,Vietnamese Restaurant,Wine Shop,Yoga Studio
0,Buffalo,0.000000,0.028148,0.028148,0.000000,0.028148,0.000000,0.000000,0.023704,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.075556,0.000000,0.000000,0.051852,0.023704,0.000000,0.000000,0.028148,0.028148,0.000000,0.000000,0.023704,0.023704,0.000000,0.000000,0.000000,0.000000,0.028148,0.023704,0.000000,0.000000,0.000000,0.028148,0.000000,0.023704,0.000000,0.023704,0.023704,0.000000,0.028148,0.028148,0.000000,0.000000,0.000000,0.028148,0.000000,0.023704,0.000000,0.000000,0.075556,0.051852,0.000000,0.000000,0.023704,0.023704,0.023704,0.000000,0.000000,0.0,0.000000,0.000000,0.023704,0.000000,0.000000,0.000000,0.000000,0.023704,0.000000,0.023704,0.028148,0.051852,0.028148
1,Chappaqua,0.066667,0.000000,0.000000,0.033333,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.033333,0.033333,0.000000,0.000000,0.000000,0.000000,0.033333,0.033333,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.033333,0.000000,0.000000,0.033333,0.000000,0.033333,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.1,0.033333,0.000000,0.000000,0.033333,0.033333,0.000000,0.033333,0.000000,0.033333,0.000000,0.000000,0.033333,0.000000
2,Flushing,0.000000,0.000000,0.000000,0.000000,0.000000,0.055556,0.000000,0.000000,0.000000,0.000000,0.055556,0.055556,0.000000,0.000000,0.055556,0.055556,0.055556,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.055556,0.055556,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.055556,0.000000,0.055556,0.000000,0.055556,0.055556,0.000000,0.111111,0.000000,0.055556,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.055556,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Hamburg,0.047619,0.000000,0.000000,0.000000,0.000000,0.047619,0.000000,0.000000,0.047619,0.047619,0.095238,0.000000,0.047619,0.000000,0.047619,0.000000,0.000000,0.000000,0.000000,0.000000,0.047619,0.000000,0.000000,0.000000,0.047619,0.000000,0.047619,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.047619,0.000000,0.000000,0.000000,0.000000,0.000000,0.047619,0.047619,0.000000,0.000000,0.000000,0.000000,0.047619,0.047619,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.142857,0.047619,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.047619,0.000000
4,Seaford,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.071429,0.000000,0.000000,0.071429,0.000000,0.071429,0.000000,0.000000,0.00000

In [ ]:
ny_grouped.shape

(5, 77)

#### Print each neighborhood along with the top 5 most common venues

In [ ]:
num_top_venues = 5

for hood in ny_grouped['city']:
    print("----"+hood+"----")
    temp = ny_grouped[ny_grouped['city'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Buffalo----
               venue  freq
0           Pharmacy  0.08
1        Coffee Shop  0.08
2          Wine Shop  0.05
3  Convenience Store  0.05
4        Pizza Place  0.05


----Chappaqua----
                 venue  freq
0                 Café  0.10
1                  Spa  0.10
2  American Restaurant  0.07
3          Pizza Place  0.07
4     Sushi Restaurant  0.03


----Flushing----
                           venue  freq
0                     Donut Shop  0.11
1                    Pizza Place  0.11
2  Paper / Office Supplies Store  0.06
3                   Tennis Court  0.06
4                    College Gym  0.06


----Hamburg----
                 venue  freq
0          Pizza Place  0.14
1                 Café  0.10
2  American Restaurant  0.05
3          Coffee Shop  0.05
4        Movie Theater  0.05


----Seaford----
                  venue  freq
0  Fast Food Restaurant  0.21
1   Sporting Goods Shop  0.07
2        Sandwich Place  0.07
3    Italian Restaurant  0.07
4          Danc

#### Let's put that into a _pandas_ datafram

In [ ]:
# function to sort the venues in descending order.

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
# create the new dataframe and display the top 10 venues for each neighborhood.

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['city']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
city_venues_sorted = pd.DataFrame(columns=columns)
city_venues_sorted['city'] = ny_grouped['city']

for ind in np.arange(ny_grouped.shape[0]):
    city_venues_sorted.iloc[ind, 1:] = return_most_common_venues(ny_grouped.iloc[ind, :], num_top_venues)

city_venues_sorted.head(10)

,city,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Buffalo,Coffee Shop,Pharmacy,Pizza Place,Wine Shop,Convenience Store,Garden Center,Dive Bar,Discount Store,Hardware Store,Juice Bar
1,Chappaqua,Café,Spa,American Restaurant,Pizza Place,Gymnastics Gym,Food,Wine Shop,Farmers Market,Jewelry Store,Deli / Bodega
2,Flushing,Donut Shop,Pizza Place,Cantonese Restaurant,College Gym,Concert Hall,Noodle House,Café,Restaurant,Museum,Tennis Court
3,Hamburg,Pizza Place,Café,American Restaurant,Coffee Shop,Wine Shop,Ice Cream Shop,Intersection,Flower Shop,Mexican Restaurant,Movie Theater
4,Seaford,Fast Food Restaurant,Sporting Goods Shop,Pizza Place,Gym,Coffee Shop,Sandwich Place,Smoke Shop,Dance Studio,Hobby Shop,Italian Restaurant


# LET'S CLUSTER THE NEIGHBORHOODS

In [ ]:
# Run _k_-means to cluster the neighborhood into 5 clusters.

# set number of clusters
kclusters = 5

ny_grouped_clustering = ny_grouped.drop('city', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(ny_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 1, 3, 4, 0], dtype=int32)

Creating a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [ ]:
# add clustering labels
city_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

ny_merged = ny_data

# merge ny_grouped with ny to add latitude/longitude for each city
ny_merged = ny_merged.join(city_venues_sorted.set_index('city'), on='city')

ny_merged.head(10) # check the last columns!

,id,address,categories,primaryCategories,city,country,latitude,longitude,menus.amountMax,menus.amountMin,menus.currency,menus.name,name,postalCode,priceRangeCurrency,priceRangeMin,priceRangeMax,province,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,AVwcxeT5ByjofQCxiCJs,371 W Ferry St,"Restaurant,Pizza Place",Accommodation & Food Services,Buffalo,US,42.915339,-78.887454,4.99,4.99,USD,2 Slices of Cheese & Pepperoni Pizza,La Nova Pizzeria,14213,USD,0,25,NY,2,Coffee Shop,Pharmacy,Pizza Place,Wine Shop,Convenience Store,Garden Center,Dive Bar,Discount Store,Hardware Store,Juice Bar
1,AVwcxeT5ByjofQCxiCJs,371 W Ferry St,"Restaurant,Pizza Place",Accommodation & Food Services,Buffalo,US,42.915339,-78.887454,15.49,15.49,USD,Garlic Lover's Pizza,La Nova Pizzeria,14213,USD,0,25,NY,2,Coffee Shop,Pharmacy,Pizza Place,Wine Shop,Convenience Store,Garden Center,Dive Bar,Discount Store,Hardware Store,Juice Bar
2,AVwcxeT5ByjofQCxiCJs,371 W Ferry St,"Restaurant,Pizza Place",Accommodation & Food Services,Buffalo,US,42.915339,-78.887454,7.99,7.99,USD,"Pizza Sub, Cheese & Pepperoni",La Nova Pizzeria,14213,USD,0,25,NY,2,Coffee Shop,Pharmacy,Pizza Place,Wine Shop,Convenience Store,Garden Center,Dive Bar,Discount Store,Hardware Store,Juice Bar
3,AVwcxeT5ByjofQCxiCJs,371 W Ferry St,"Restaurant,Pizza Place",Accommodation & Food Services,Buffalo,US,42.915339,-78.887454,16.99,16.99,USD,Chicken Bacon Ranch Pizza,La Nova Pizzeria,14213,USD,0,25,NY,2,Coffee Shop,Pharmacy,Pizza Place,Wine Shop,Convenience Store,Garden Center,Dive Bar,Discount Store,Hardware Store,Juice Bar
4,AVwcxeT5ByjofQCxiCJs,371 W Ferry St,"Restaurant,Pizza Place",Accommodation & Food Services,Buffalo,US,42.915339,-78.887454,2.70,2.70,USD,La Nova Original White Pizza Nova,La Nova Pizzeria,14213,USD,0,25,NY,2,Coffee Shop,Pharmacy,Pizza Place,Wine Shop,Convenience Store,Garden Center,Dive Bar,Discount Store,Hardware Store,Juice Bar
5,AVwcxeT5ByjofQCxiCJs,371 W Ferry St,"Restaurant,Pizza Place",Accommodation & Food Services,Buffalo,US,42.915339,-78.887454,0.00,0.00,USD,Chicken Wing Pizza,La Nova Pizzeria,14213,USD,0,25,NY,2,Coffee Shop,Pharmacy,Pizza Place,Wine Shop,Convenience Store,Garden Center,Dive Bar,Discount Store,Hardware Store,Juice Bar
6,AVwcxeT5ByjofQCxiCJs,371 W Ferry St,"Restaurant,Pizza Place",Accommodation & Food Services,Buffalo,US,42.915339,-78.887454,12.25,12.25,USD,Garlic Loverampaposs Pizza,La Nova Pizzeria,14213,USD,0,25,NY,2,Coffee Shop,Pharmacy,Pizza Place,Wine Shop,Convenience Store,Garden Center,Dive Bar,Discount Store,Hardware Store,Juice Bar
7,AVwcxeT5ByjofQCxiCJs,371 W Ferry St,"Restaurant,Pizza Place",Accommodation & Food Services,Buffalo,US,42.915339,-78.887454,16.49,16.49,USD,Chicken Finger Pizza,La Nova Pizzeria,14213,USD,0,25,NY,2,Coffee Shop,Pharmacy,Pizza Place,Wine Shop,Convenience Store,Garden Center,Dive Bar,Discount Store,Hardware Store,Juice Bar
8,AVwcxeT5ByjofQCxiCJs,371 W Ferry St,"Restaurant,Pizza Place",Accommodation & Food Services,Buffalo,US,42.915339,-78.887454,0.00,0.00,USD,Meat Lovers Pizza,La Nova Pizzeria,14213,USD,0,25,NY,2,Coffee Shop,Pharmacy,Pizza Place,Wine Shop,Convenience Store,Garden Center,Dive Bar,Discount Store,Hardware Store,Juice Bar
9,AVwcxeT5ByjofQCxiCJs,371 W Ferry St,"Restaurant,Pizza Place",Accommodation & Food Services,Buffalo,US,42.915339,-78.887454,12.40,12.40,USD,Hawaiian Pizza,La Nova Pizzeria,14213,USD,0,25,NY,2,Coffee Shop,Pharmacy,Pizza Place,Wine Shop,Convenience Store,Garden Center,Dive Bar,Discount Store,Hardware Store,Juice Bar


In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ny_merged['latitude'], ny_merged['longitude'], ny_merged['city'], ny_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# EXAMINE CLUSTERS 

CLUSTER ONE

In [ ]:
buffalo = ny_merged.loc[ny_merged['Cluster Labels'] == 0, ny_merged.columns[[1] + list(range(5, ny_merged.shape[1]))]]
buffalo

,address,country,latitude,longitude,menus.amountMax,menus.amountMin,menus.currency,menus.name,name,postalCode,priceRangeCurrency,priceRangeMin,priceRangeMax,province,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
103,3842 Sunrise Hwy,US,40.674464,-73.490065,10.50,10.50,USD,Whole Wheat Personal Cheese Pizza,Mario's Pizzeria,11783,USD,0,25,NY,0,Fast Food Restaurant,Sporting Goods Shop,Pizza Place,Gym,Coffee Shop,Sandwich Place,Smoke Shop,Dance Studio,Hobby Shop,Italian Restaurant
104,3842 Sunrise Hwy,US,40.674464,-73.490065,16.75,16.75,USD,Sicilian Cheese Pizza,Mario's Pizzeria,11783,USD,0,25,NY,0,Fast Food Restaurant,Sporting Goods Shop,Pizza Place,Gym,Coffee Shop,Sandwich Place,Smoke Shop,Dance Studio,Hobby Shop,Italian Restaurant
105,3842 Sunrise Hwy,US,40.674464,-73.490065,13.50,13.50,USD,Whole Wheat Personal 1 Topping Pizza,Mario's Pizzeria,11783,USD,0,25,NY,0,Fast Food Restaurant,Sporting Goods Shop,Pizza Place,Gym,Coffee Shop,Sandwich Place,Smoke Shop,Dance Studio,Hobby Shop,Italian Restaurant
106,3842 Sunrise Hwy,US,40.674464,-73.490065,20.00,20.00,USD,White Broccoli or Spinach Pizza,Mario's Pizzeria,11783,USD,0,25,NY,0,Fast Food Restaurant,Sporting Goods Shop,Pizza Place,Gym,Coffee Shop,Sandwich Place,Smoke Shop,Dance Studio,Hobby Shop,Italian Restaurant
107,3842 Sunrise Hwy,US,40.674464,-73.490065,3.85,3.85,USD,Vegetable Pizza,Mario's Pizzeria,11783,USD,0,25,NY,0,Fast Food Restaurant,Sporting Goods Shop,Pizza Place,Gym,Coffee Shop,Sandwich Place,Smoke Shop,Dance Studio,Hobby Shop,Italian Restaurant
108,3842 Sunrise Hwy,US,40.674464,-73.490065,4.85,4.85,USD,Meat Stuffed Pizza,Mario's Pizzeria,11783,USD,0,25,NY,0,Fast Food Restaurant,Sporting Goods Shop,Pizza Place,Gym,Coffee Shop,Sandwich Place,Smoke Shop,Dance Studio,Hobby Shop,Italian Restaurant
109,3842 Sunrise Hwy,US,40.674464,-73.490065,3.75,3.75,USD,White Spinach Pizza,Mario's Pizzeria,11783,USD,0,25,NY,0,Fast Food Restaurant,Sporting Goods Shop,Pizza Place,Gym,Coffee Shop,Sandwich Place,Smoke Shop,Dance Studio,Hobby Shop,Italian Restaurant
110,3842 Sunrise Hwy,US,40.674464,-73.490065,2.75,2.75,USD,Marinara Pizza,Mario's Pizzeria,11783,USD,0,25,NY,0,Fast Food Restaurant,Sporting Goods Shop,Pizza Place,Gym,Coffee Shop,Sandwich Place,Smoke Shop,Dance Studio,Hobby Shop,Italian Restaurant
111,3842 Sunrise Hwy,US,40.674464,-73.490065,9.00,9.00,USD,Pasta Pizza,Mario's Pizzeria,11783,USD,0,25,NY,0,Fast Food Restaurant,Sporting Goods Shop,Pizza Place,Gym,Coffee Shop,Sandwich Place,Smoke Shop,Dance Studio,Hobby Shop,Italian Restaurant
112,3842 Sunrise Hwy,US,40.674464,-73.490065,4.00,4.00,USD,Chicken Caesar Salad Pizza,Mario's Pizzeria,11783,USD,0,25,NY,0,Fast Food Restaurant,Sporting Goods Shop,Pizza Place,Gym,Coffee Shop,Sandwich Place,Smoke Shop,Dance Studio,Hobby Shop,Italian Restaurant


CLUSTER TWO

In [ ]:
flushing = ny_merged.loc[ny_merged['Cluster Labels'] == 0, ny_merged.columns[[1] + list(range(5, ny_merged.shape[1]))]]
flushing

,address,country,latitude,longitude,menus.amountMax,menus.amountMin,menus.currency,menus.name,name,postalCode,priceRangeCurrency,priceRangeMin,priceRangeMax,province,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
103,3842 Sunrise Hwy,US,40.674464,-73.490065,10.50,10.50,USD,Whole Wheat Personal Cheese Pizza,Mario's Pizzeria,11783,USD,0,25,NY,0,Fast Food Restaurant,Sporting Goods Shop,Pizza Place,Gym,Coffee Shop,Sandwich Place,Smoke Shop,Dance Studio,Hobby Shop,Italian Restaurant
104,3842 Sunrise Hwy,US,40.674464,-73.490065,16.75,16.75,USD,Sicilian Cheese Pizza,Mario's Pizzeria,11783,USD,0,25,NY,0,Fast Food Restaurant,Sporting Goods Shop,Pizza Place,Gym,Coffee Shop,Sandwich Place,Smoke Shop,Dance Studio,Hobby Shop,Italian Restaurant
105,3842 Sunrise Hwy,US,40.674464,-73.490065,13.50,13.50,USD,Whole Wheat Personal 1 Topping Pizza,Mario's Pizzeria,11783,USD,0,25,NY,0,Fast Food Restaurant,Sporting Goods Shop,Pizza Place,Gym,Coffee Shop,Sandwich Place,Smoke Shop,Dance Studio,Hobby Shop,Italian Restaurant
106,3842 Sunrise Hwy,US,40.674464,-73.490065,20.00,20.00,USD,White Broccoli or Spinach Pizza,Mario's Pizzeria,11783,USD,0,25,NY,0,Fast Food Restaurant,Sporting Goods Shop,Pizza Place,Gym,Coffee Shop,Sandwich Place,Smoke Shop,Dance Studio,Hobby Shop,Italian Restaurant
107,3842 Sunrise Hwy,US,40.674464,-73.490065,3.85,3.85,USD,Vegetable Pizza,Mario's Pizzeria,11783,USD,0,25,NY,0,Fast Food Restaurant,Sporting Goods Shop,Pizza Place,Gym,Coffee Shop,Sandwich Place,Smoke Shop,Dance Studio,Hobby Shop,Italian Restaurant
108,3842 Sunrise Hwy,US,40.674464,-73.490065,4.85,4.85,USD,Meat Stuffed Pizza,Mario's Pizzeria,11783,USD,0,25,NY,0,Fast Food Restaurant,Sporting Goods Shop,Pizza Place,Gym,Coffee Shop,Sandwich Place,Smoke Shop,Dance Studio,Hobby Shop,Italian Restaurant
109,3842 Sunrise Hwy,US,40.674464,-73.490065,3.75,3.75,USD,White Spinach Pizza,Mario's Pizzeria,11783,USD,0,25,NY,0,Fast Food Restaurant,Sporting Goods Shop,Pizza Place,Gym,Coffee Shop,Sandwich Place,Smoke Shop,Dance Studio,Hobby Shop,Italian Restaurant
110,3842 Sunrise Hwy,US,40.674464,-73.490065,2.75,2.75,USD,Marinara Pizza,Mario's Pizzeria,11783,USD,0,25,NY,0,Fast Food Restaurant,Sporting Goods Shop,Pizza Place,Gym,Coffee Shop,Sandwich Place,Smoke Shop,Dance Studio,Hobby Shop,Italian Restaurant
111,3842 Sunrise Hwy,US,40.674464,-73.490065,9.00,9.00,USD,Pasta Pizza,Mario's Pizzeria,11783,USD,0,25,NY,0,Fast Food Restaurant,Sporting Goods Shop,Pizza Place,Gym,Coffee Shop,Sandwich Place,Smoke Shop,Dance Studio,Hobby Shop,Italian Restaurant
112,3842 Sunrise Hwy,US,40.674464,-73.490065,4.00,4.00,USD,Chicken Caesar Salad Pizza,Mario's Pizzeria,11783,USD,0,25,NY,0,Fast Food Restaurant,Sporting Goods Shop,Pizza Place,Gym,Coffee Shop,Sandwich Place,Smoke Shop,Dance Studio,Hobby Shop,Italian Restaurant


## Results and Discussion <a name="results"></a>

My analysis shows that the two states that have the most pizza locations by a significant higher amount are NY and CA. So, I decided on just using those two states and doing an West coast vs an East coast thing.

Then looked at the most popular pizza locaton names and the type of pizzas that are served the most. It looked like New Yorkers enjoy their pizza the most becasue Ny was number one when it comes to the most pizza locations. Which is why I went into futher analysis with NY to look at the cities within NY.

Flushing and Hanburg came back with the most data from foursqaure. Which is why I decided to cluster NY and those cities. 

## Conclusion <a name="conclusion"></a>

Purpose of this project was to establish which states around the US provided the most pizza locations for the pizza lovers. Then determine which state would be best for the pizza lovers who are also looking for other thigs to do will visiting those states. 

Different decsiions can be made in which states pizza lovers would want to visit and or city they would like to stay in within that state. Which will be determined by the piza lovers important factors such as entertainment, bars, other food places, lq stores, gyms etc. 